In [ ]:
import numpy as np
import pandas as pd
import os
import time
import string
import random

import pickle

In [ ]:
data_f = '[the output from convert4training]'
PATH ='[output path]'

In [ ]:
with open(data_f, 'rb') as f:
    data = pickle.load(f)

In [ ]:
seqs = {}
labels = {}
times = {}
for k,v in data.items():
    labels[k] = v[0]
    seqs[k] = v[1][0]
    times[k] = v[1][1]

### count case and control numbers

In [ ]:
n_p = 0
n_n = 0
for k,v in labels.items():
    if v == 0:
        n_n += 1
    if v == 1:
        n_p += 1
print(n_p)
print(n_n)

### count number of visits

In [ ]:
n_v_p = []
n_v_n = []
for k,v in labels.items():
    n_v = len(seqs[k])
    if v == 0:
        n_v_n.append(n_v)
    if v == 1:
        n_v_p.append(n_v)
print(np.mean(n_v_p))
print(np.std(n_v_p))

print(np.mean(n_v_n))
print(np.std(n_v_n))

### count number of diagnosis, medication, surgical, labtest and clinical events

In [ ]:
n_med_p = []
n_med_n = []
for k,v in labels.items():
    tmp = 0
    for s in seqs[k]:
        tmp += np.sum([str(code).startswith('EVENT_') for code in s])
    if 1:
        if v == 0:
            n_med_n.append(tmp)
        if v == 1:
            n_med_p.append(tmp)
print(np.mean(n_med_p))
print(np.std(n_med_p))

print(np.mean(n_med_n))
print(np.std(n_med_n))

In [ ]:
n_dx_p = []
n_dx_n = []
for k,v in labels.items():
    tmp = 0
    for s in seqs[k]:
        tmp += np.sum([str(code).startswith('_') for code in s])
    if 1:
        if v == 0:
            n_dx_n.append(tmp)
        if v == 1:
            n_dx_p.append(tmp)
print(np.mean(n_dx_p))
print(np.std(n_dx_p))

print(np.mean(n_dx_n))
print(np.std(n_dx_n))

In [ ]:
new_seqs = {}
for psk, visits in seqs.items():
    new_visits = []
    for visit in visits:
        agecode = [code for code in visit if str(code).startswith('A_')]
        gendercode = [code for code in visit if str(code).startswith('GEND_')]
        diags = [code for code in visit if str(code).startswith('DIAG_')]
        up_diags = [code.split('.')[0] for code in diags]
        new_visit = up_diags + agecode + gendercode
        if len(new_visit) >= 1:
            new_visits.append(list(set(new_visit)))
    new_seqs[psk]=new_visits

In [ ]:
old_seqs = seqs
seqs = new_seqs

In [ ]:
### remove samples with only one visit
new_labels = {k:labels[k] for k, visits in seqs.items() if len(visits)>=2}

In [ ]:
len(new_labels)

In [ ]:
flat_list = []
new_seqs = {}
for k, v in new_labels.items():
    new_visits = []
    for visit in seqs[k]:
        uniqs = list(set(visit))
        flat_list.extend(uniqs)
        new_visits.append(uniqs)
    new_seqs[k] = new_visits

In [ ]:
import numpy as np
allcodes = np.array(flat_list)
unique, counts = np.unique(allcodes, return_counts=True)
keys_fre = dict(zip(unique, counts))

In [ ]:
index = 1
keys_idx = {}
for k, ct in keys_fre.items():
    if ct>10:
        keys_idx[k] = index
        index += 1
    else:
        keys_idx[k] = 0

In [ ]:
index

In [ ]:
if not os.path.exists(PATH):
    os.makedirs(PATH)

### split the dataset into training, evaluating and testing sets

In [ ]:
psks = [k for k,v in new_labels.items()]
val_ratio = 0.2
test_ratio = 0.1
val_samples = int(len(psks) * val_ratio)
test_samples = int(len(psks) * test_ratio)
random.seed( 7747 )
random.shuffle(psks)
val_psks = psks[0:val_samples]
test_psks = psks[val_samples:val_samples+test_samples]
train_psks = psks[val_samples+test_samples:]

In [ ]:
def code2idx(visits):
    res=[]
    for v in visits:
        new_v = []
        for code in v:
            #print(code)
            idx = keys_idx.get(code)
            if idx is None:
                idx = nodes_dict.get('NULL')
            new_v.append(idx)
        res.append(new_v)
    return res

In [ ]:
seqs_valid = [code2idx(new_seqs[psk]) for psk in val_psks]
seqs_test = [code2idx(new_seqs[psk]) for psk in test_psks]
seqs_train = [code2idx(new_seqs[psk]) for psk in train_psks]

In [ ]:
labels_valid = [new_labels[psk] for psk in val_psks]
labels_test = [new_labels[psk] for psk in test_psks]
labels_train = [new_labels[psk] for psk in train_psks]

In [ ]:
times_valid = [times[psk] for psk in val_psks]
times_test = [times[psk] for psk in test_psks]
times_train = [times[psk] for psk in train_psks]

In [ ]:
seqs_all = [code2idx(new_seqs[psk]) for psk in psks]
labels_all = [new_labels[psk] for psk in psks]

In [ ]:
TRAINPATH=os.path.join(PATH, 'train.default')
if not os.path.exists(TRAINPATH):
    os.makedirs(TRAINPATH)

In [ ]:
with open(os.path.join(PATH, 'seqs.valid'), 'wb') as f:
    pickle.dump(seqs_valid, f, protocol=2)

with open(os.path.join(PATH, 'seqs.test'), 'wb') as f:
    pickle.dump(seqs_test, f, protocol=2)
with open(os.path.join(TRAINPATH, 'seqs.train'), 'wb') as f:
    pickle.dump(seqs_train, f, protocol=2)

In [ ]:
with open(os.path.join(PATH, 'labels.valid'), 'wb') as f:
    pickle.dump(labels_valid, f, protocol=2)

with open(os.path.join(PATH, 'labels.test'), 'wb') as f:
    pickle.dump(labels_test, f, protocol=2)
with open(os.path.join(TRAINPATH, 'labels.train'), 'wb') as f:
    pickle.dump(labels_train, f, protocol=2)

In [ ]:
with open(os.path.join(PATH, 'times.valid'), 'wb') as f:
    pickle.dump(times_valid, f, protocol=2)

with open(os.path.join(PATH, 'times.test'), 'wb') as f:
    pickle.dump(times_test, f, protocol=2)
with open(os.path.join(TRAINPATH, 'times.train'), 'wb') as f:
    pickle.dump(times_train, f, protocol=2)

In [ ]:
with open(os.path.join(PATH, 'psk.valid'), 'wb') as f:
    pickle.dump(val_psks, f, protocol=2)

with open(os.path.join(PATH, 'psk.test'), 'wb') as f:
    pickle.dump(test_psks, f, protocol=2)
with open(os.path.join(TRAINPATH, 'psk.train'), 'wb') as f:
    pickle.dump(train_psks, f, protocol=2)

In [ ]:
with open(os.path.join(PATH, 'seqs.all'), 'wb') as f:
    pickle.dump(seqs_all, f, protocol=2)
with open(os.path.join(PATH, 'labels.all'), 'wb') as f:
    pickle.dump(labels_all, f, protocol=2)
with open(os.path.join(PATH, 'psk.all'), 'wb') as f:
    pickle.dump(psks, f, protocol=2)


In [ ]:
with open(os.path.join(PATH, 'codesize'), 'w') as f:
    f.write(f'{index}\n')

In [ ]:
with open(os.path.join(PATH, 'keys.mapping'), 'wb') as f:
          pickle.dump(keys_idx, f, protocol=2)

### upsampling the positive samples

In [ ]:
default_TRAINPATH = os.path.join(PATH, 'train.default')
with open(os.path.join(default_TRAINPATH, 'psk.train'), 'rb') as f:
    train_psks = pickle.load(f)

In [ ]:
p_train_psks = [k for k in train_psks if new_labels[k]==1]
n_train_psks = [k for k in train_psks if new_labels[k]==0]
print(len(p_train_psks))
print(len(n_train_psks))

In [ ]:
n_new_train_psks = n_train_psks

In [ ]:
len(n_new_train_psks)

In [ ]:
new_train_psks = []
new_train_psks.extend(p_train_psks)
new_train_psks.extend(p_train_psks)
new_train_psks.extend(p_train_psks)
new_train_psks.extend(p_train_psks)

In [ ]:
len(new_train_psks)

In [ ]:
new_train_psks.extend(n_new_train_psks)

In [ ]:
len(new_train_psks)

In [ ]:
random.seed(7747)
random.shuffle(new_train_psks)

In [ ]:
new_seqs_train = [code2idx(new_seqs[psk]) for psk in new_train_psks]

In [ ]:
new_labels_train = [new_labels[psk] for psk in new_train_psks]

In [ ]:
new_times_train = [times[psk] for psk in new_train_psks]

In [ ]:
AUG_TRAINPATH=os.path.join(PATH, 'train.positive_aug_3')
if not os.path.exists(AUG_TRAINPATH):
    os.makedirs(AUG_TRAINPATH)

In [ ]:
with open(os.path.join(AUG_TRAINPATH, 'seqs.train'), 'wb') as f:
    pickle.dump(new_seqs_train, f, protocol=2)

In [ ]:
with open(os.path.join(AUG_TRAINPATH, 'labels.train'), 'wb') as f:
    pickle.dump(new_labels_train, f, protocol=2)

In [ ]:
with open(os.path.join(AUG_TRAINPATH, 'times.train'), 'wb') as f:
    pickle.dump(new_times_train, f, protocol=2)


In [ ]:
with open(os.path.join(AUG_TRAINPATH, 'psk.train'), 'wb') as f:
    pickle.dump(new_train_psks, f, protocol=2)